# Bonus Lab — Reconstructing a DC Offset
## Strip the DC, set a new one, and understand why every amplifier stage does this

---

**Prerequisites:** Module 00 (voltage dividers, AC vs DC), Bonus Lab 00 (AC coupling circuit)

**What you'll learn:**
- How to combine AC coupling with a resistor divider to place an AC signal at any DC level
- Why a capacitor alone can only *remove* DC — it can't create a voltage from nothing
- The loading tradeoff: divider resistance vs signal attenuation
- How clamper circuits (capacitor + diode) shift DC without a divider
- Where re-biasing appears in real circuits: amplifier stages, ADC inputs, audio, and more

**Equipment:** Fnirsi 2C53T oscilloscope + signal generator, Klein MM300 multimeter

**Parts from your kit:**
- 1µF capacitor
- 10kΩ resistors (×3 or ×4)
- 100kΩ resistor (for exploration)

**Estimated time:** 30–45 minutes

In [ ]:
# Setup — run this cell first
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig
%matplotlib inline

plt.rcParams.update({
    'figure.figsize': (10, 5),
    'axes.grid': True,
    'font.size': 12,
    'lines.linewidth': 2,
    'grid.alpha': 0.3,
})
print("Setup complete.")

---
## 1. The Problem

Bonus Lab 00 showed how AC coupling works: a series capacitor blocks DC, a resistor to ground sets the output at 0V, and the AC signal rides on top. The Fnirsi's 0–3V unipolar output becomes a ±1.5V bipolar signal.

But **0V isn't always where you want the signal.** What if you need it centered on 2.5V? Or 1.65V?

This comes up constantly in electronics:
- **ADC input conditioning:** A 0–3.3V sensor output needs to be centered at 1.65V for a differential ADC
- **Amplifier biasing:** Each transistor amplifier stage needs its input at a specific DC voltage to operate in its linear region
- **Level shifting:** Interfacing between circuits that run at different supply voltages

A capacitor alone can only *remove* DC — it can't create a voltage from nothing. To *set* a new DC level, you need a DC source. The simplest DC source is a **resistor divider** connected to a supply rail.

The recipe: **AC-couple (strip the old DC), then re-bias with a resistor divider (set the new DC).** The AC signal rides on top of whatever DC level the divider establishes.

---
## 2. The Circuit: AC Couple + Resistor Divider

```
                     C
Signal In ─────┤├─────────┬───── Signal Out
              1µF         │
                       ┌──┴──┐
                       │     │
                      R1    R2
                       │     │
                      V+    GND
```

How it works:

1. **The capacitor blocks the original DC.** Whatever DC offset the input signal had is stripped away. Only the AC component passes through.

2. **R1 and R2 form a voltage divider** from the supply voltage V+ to ground. They set the DC voltage at the output node:

$$V_{bias} = V_{+} \times \frac{R_2}{R_1 + R_2}$$

3. **The AC signal rides on top of V_bias.** The output is:

$$v_{out}(t) = V_{bias} + v_{AC}(t)$$

Two impedance paths meet at the output node:
- **AC path:** Through the capacitor (low impedance at signal frequencies)
- **DC path:** Through the divider (sets the bias point)

At DC, the capacitor is an open circuit — only the divider determines the voltage. At signal frequencies, the capacitor is a low impedance — the AC signal passes through with minimal loss. This is just **superposition**: the AC and DC contributions are independent.

---
## 3. Voltage Divider Refresher

A voltage divider is two resistors in series, with the output taken from the middle:

$$V_{out} = V_{supply} \times \frac{R_2}{R_1 + R_2}$$

where R1 connects to V_supply and R2 connects to ground.

Quick reference for common bias points:

| V_supply | R1 | R2 | V_bias | Use case |
|----------|----|----|--------|----------|
| 5V | 10kΩ | 10kΩ | 2.5V | V_supply/2 — most common |
| 3.3V | 10kΩ | 10kΩ | 1.65V | 3.3V rail midpoint |
| 5V | 40kΩ | 10kΩ | 1.0V | 1/5 of supply |
| 5V | 10kΩ | 20kΩ | 3.33V | 2/3 of supply |

Equal resistors always give half the supply voltage, regardless of the resistor value. The value of the resistors matters for other reasons (loading, power consumption) — more on that later.

In [ ]:
# --- Simulation: stripping and re-biasing a signal ---

f = 1000  # Hz
t = np.linspace(0, 3 / f, 3000)  # 3 cycles

# Original signal: Fnirsi-like 0-3V sine (DC = 1.5V, AC = ±1.5V)
V_dc_original = 1.5
V_ac_peak = 1.5
v_original = V_dc_original + V_ac_peak * np.sin(2 * np.pi * f * t)

# Step 1: AC couple — strip DC, signal is ±1.5V around 0V
v_ac_coupled = v_original - V_dc_original  # idealized: cap removes DC perfectly

# Step 2: Re-bias at 2.5V with a divider (5V supply, equal 10kΩ resistors)
V_bias = 2.5
v_rebiased = V_bias + v_ac_coupled

# 3-panel plot
fig, axes = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

axes[0].plot(t * 1000, v_original, color='#2980b9')
axes[0].axhline(y=V_dc_original, color='orange', linestyle='--', alpha=0.7,
                label=f'DC = {V_dc_original}V')
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title('Original: Fnirsi output (0V to 3V)')
axes[0].set_ylim(-2, 5)
axes[0].legend(loc='upper right')

axes[1].plot(t * 1000, v_ac_coupled, color='#27ae60')
axes[1].axhline(y=0, color='orange', linestyle='--', alpha=0.7,
                label='DC = 0V')
axes[1].set_ylabel('Voltage (V)')
axes[1].set_title('After AC coupling: DC stripped (±1.5V around 0V)')
axes[1].set_ylim(-2, 5)
axes[1].legend(loc='upper right')

axes[2].plot(t * 1000, v_rebiased, color='#e74c3c')
axes[2].axhline(y=V_bias, color='orange', linestyle='--', alpha=0.7,
                label=f'DC = {V_bias}V (from divider)')
axes[2].set_ylabel('Voltage (V)')
axes[2].set_xlabel('Time (ms)')
axes[2].set_title(f'After re-biasing: AC rides on {V_bias}V ({V_bias - V_ac_peak}V to {V_bias + V_ac_peak}V)')
axes[2].set_ylim(-2, 5)
axes[2].legend(loc='upper right')

plt.tight_layout()
plt.show()

print(f"Original signal:  {V_dc_original - V_ac_peak}V to {V_dc_original + V_ac_peak}V  (DC = {V_dc_original}V)")
print(f"AC coupled:       {-V_ac_peak}V to {V_ac_peak}V  (DC = 0V)")
print(f"Re-biased:        {V_bias - V_ac_peak}V to {V_bias + V_ac_peak}V  (DC = {V_bias}V)")
print(f"\nSame AC component in all three. Only the DC level changed.")

---
## 4. The Math — Why It Adds

This works because of **superposition**: in a linear circuit, the response to multiple sources is the sum of the responses to each source individually.

At the output node, there are two "sources":

1. **The AC signal** coming through the capacitor
2. **The DC bias** coming from the resistor divider

Analyze each independently:

- **At DC (0 Hz):** The capacitor is an open circuit. No current flows from the signal source. The only voltage at the output comes from the divider: $V_{out} = V_{bias}$.

- **At signal frequencies:** The capacitor has low impedance ($X_C = 1/(2\pi f C)$). The AC signal passes through with minimal loss. The supply voltage is a constant (AC ground), so the divider contributes nothing at AC — it just looks like $R_1 \| R_2$ to ground.

Add them together:

$$v_{out}(t) = V_{bias} + v_{AC}(t)$$

The AC component rides on top of whatever DC the divider establishes. You can set any DC level you want by choosing the right resistor ratio — the AC signal doesn't care.

---
## 5. The Loading Tradeoff

The divider resistors have to satisfy two competing requirements:

**Large enough** not to attenuate the AC signal. The divider's parallel resistance ($R_1 \| R_2$) appears as a load on the AC signal. If this resistance is comparable to the capacitor's impedance at the signal frequency, it forms a voltage divider *with the capacitor* and reduces the AC amplitude.

**Small enough** to provide a stiff bias point. If the resistors are very large, even small currents drawn by the load can shift the DC voltage away from the intended value.

Rule of thumb:
- Divider impedance ($R_1 \| R_2$) should be **much less than** the load impedance (so the bias doesn't sag)
- Divider impedance should be **much greater than** the capacitor's impedance at the signal frequency (so AC isn't attenuated)

With our components:
- Two 10kΩ resistors: $R_1 \| R_2 = 5\text{kΩ}$
- Capacitor impedance at 1 kHz: $X_C = 1/(2\pi \times 1000 \times 1\mu\text{F}) \approx 159\Omega$
- Scope input impedance: 1MΩ

So: $X_C$ (159Ω) ≪ $R_{divider}$ (5kΩ) ≪ $R_{load}$ (1MΩ). This is a well-designed circuit — the AC signal passes through with minimal loss, and the bias point is rock-solid into the high-impedance scope input.

In [ ]:
# --- Simulation: loading effects on AC amplitude ---
# The AC signal sees the divider's parallel resistance as a load.
# AC gain = R_parallel / sqrt(R_parallel^2 + Xc^2)

f_sig = 1000  # Hz
C = 1e-6      # 1µF
Xc = 1 / (2 * np.pi * f_sig * C)  # ~159Ω

# Sweep divider resistance (equal resistors, so R_parallel = R/2)
R_values = np.logspace(1, 6, 500)  # 10Ω to 1MΩ per resistor
R_parallel = R_values / 2  # for equal resistors

# AC voltage divider between Xc and R_parallel
ac_gain = R_parallel / np.sqrt(R_parallel**2 + Xc**2)

fig, ax = plt.subplots(figsize=(12, 6))
ax.semilogx(R_values / 1e3, ac_gain * 100, color='#2980b9', linewidth=2)
ax.axhline(y=90, color='orange', linestyle='--', alpha=0.5, label='90% threshold')
ax.axhline(y=99, color='green', linestyle='--', alpha=0.5, label='99% threshold')

# Mark our 10kΩ resistors
R_ours = 10e3
R_par_ours = R_ours / 2
gain_ours = R_par_ours / np.sqrt(R_par_ours**2 + Xc**2)
ax.plot(R_ours / 1e3, gain_ours * 100, 'ro', markersize=10, zorder=5,
        label=f'Our circuit: 10kΩ ({gain_ours*100:.1f}%)')

# Mark small resistors (bad)
R_small = 100
R_par_small = R_small / 2
gain_small = R_par_small / np.sqrt(R_par_small**2 + Xc**2)
ax.plot(R_small / 1e3, gain_small * 100, 'rs', markersize=10, zorder=5,
        label=f'100Ω resistors ({gain_small*100:.1f}%) — too small!')

ax.set_xlabel('Divider resistor value (kΩ)')
ax.set_ylabel('AC amplitude at output (% of input)')
ax.set_title(f'AC Signal Preservation vs Divider Resistance (C = 1µF, f = {f_sig} Hz)')
ax.set_ylim(0, 105)
ax.legend(fontsize=10)

plt.tight_layout()
plt.show()

print(f"Capacitor impedance at {f_sig} Hz: Xc = {Xc:.1f}Ω")
print(f"")
print(f"{'Divider R':<12} {'R_parallel':<12} {'AC gain':>10} {'AC V_pp (of 3V)':>16}")
print("-" * 52)
for R in [100, 330, 1e3, 3.3e3, 10e3, 47e3, 100e3]:
    Rp = R / 2
    g = Rp / np.sqrt(Rp**2 + Xc**2)
    label = f"{R/1e3:.1f}kΩ" if R >= 1e3 else f"{R:.0f}Ω"
    print(f"{label:<12} {Rp:.0f}Ω{'':<{8-len(f'{Rp:.0f}')}} {g:>9.1%} {g*3:>14.2f}V")

In [ ]:
# --- Simulation: time-domain view of loading at different divider resistances ---

f = 1000
C = 1e-6
V_bias = 2.5
V_ac_peak = 1.5
dt = 1e-6
t = np.arange(0, 5e-3, dt)  # 5 ms = 5 cycles

# Input: AC component only (post-coupling-cap, idealized)
v_ac_in = V_ac_peak * np.sin(2 * np.pi * f * t)

divider_configs = [
    (100,   '#e74c3c', '100Ω (too small — attenuates AC)'),
    (1e3,   '#e67e22', '1kΩ (marginal)'),
    (10e3,  '#27ae60', '10kΩ (our circuit — good)'),
    (100e3, '#2980b9', '100kΩ (fine for high-Z load)'),
]

fig, axes = plt.subplots(len(divider_configs), 1, figsize=(12, 10), sharex=True)

for ax, (R, color, label) in zip(axes, divider_configs):
    R_par = R / 2  # parallel resistance of equal divider
    Xc = 1 / (2 * np.pi * f * C)
    # AC gain through the cap + divider load
    ac_gain = R_par / np.sqrt(R_par**2 + Xc**2)
    
    v_out = V_bias + ac_gain * v_ac_in
    
    ax.plot(t * 1000, v_out, color=color, linewidth=1.5)
    ax.axhline(y=V_bias, color='orange', linestyle='--', alpha=0.5)
    ax.set_ylabel('V')
    ax.set_title(f'R = {label}  —  AC amplitude: {ac_gain*100:.1f}% of input  '
                 f'(V_pp = {2*ac_gain*V_ac_peak:.2f}V)', fontsize=11)
    ax.set_ylim(0, 5)

axes[-1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.show()

print("With 100Ω resistors, the divider's parallel resistance (50Ω) is comparable")
print(f"to Xc ({1/(2*np.pi*f*C):.0f}Ω), so the AC is heavily attenuated.")
print("With 10kΩ or larger, the AC passes through nearly intact.")

---
## 6. Choosing a Bias Voltage — Worked Examples

### Example 1: Center on V_supply/2 (most common)

**Goal:** Bias a signal to 2.5V from a 5V supply.

Use equal resistors: $R_1 = R_2 = 10\text{kΩ}$

$$V_{bias} = 5\text{V} \times \frac{10\text{k}}{10\text{k} + 10\text{k}} = 2.5\text{V}$$

This is the most common case. Half-supply biasing maximizes the voltage swing in both directions before clipping.

### Example 2: Center on 1.65V from 3.3V

Same idea — equal resistors give half the supply:

$$V_{bias} = 3.3\text{V} \times \frac{10\text{k}}{10\text{k} + 10\text{k}} = 1.65\text{V}$$

### Example 3: Bias to 1.0V from 5V

Need $V_{bias}/V_{supply} = 1/5$, so $R_2/(R_1 + R_2) = 1/5$, giving $R_1 = 4 \times R_2$.

Use $R_1 = 40\text{kΩ}$ (four 10kΩ in series) and $R_2 = 10\text{kΩ}$:

$$V_{bias} = 5\text{V} \times \frac{10\text{k}}{40\text{k} + 10\text{k}} = 1.0\text{V}$$

In [ ]:
# --- Visualize the three bias examples ---

f = 1000
t = np.linspace(0, 3 / f, 3000)
V_ac_peak = 1.5  # AC component from Fnirsi
v_ac = V_ac_peak * np.sin(2 * np.pi * f * t)

examples = [
    (5.0, 10e3, 10e3, '5V supply, R1=R2=10kΩ'),
    (3.3, 10e3, 10e3, '3.3V supply, R1=R2=10kΩ'),
    (5.0, 40e3, 10e3, '5V supply, R1=40kΩ, R2=10kΩ'),
]

fig, axes = plt.subplots(3, 1, figsize=(12, 9), sharex=True)
colors = ['#2980b9', '#27ae60', '#e74c3c']

for ax, (V_supply, R1, R2, label), color in zip(axes, examples, colors):
    V_bias = V_supply * R2 / (R1 + R2)
    v_out = V_bias + v_ac
    
    ax.plot(t * 1000, v_out, color=color, linewidth=2)
    ax.axhline(y=V_bias, color='orange', linestyle='--', alpha=0.7,
               label=f'V_bias = {V_bias:.2f}V')
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.set_ylabel('Voltage (V)')
    ax.set_title(f'{label}  →  V_bias = {V_bias:.2f}V  '
                 f'(output: {V_bias-V_ac_peak:.2f}V to {V_bias+V_ac_peak:.2f}V)',
                 fontsize=11)
    ax.set_ylim(-2, 5.5)
    ax.legend(loc='upper right')

axes[-1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.show()

print("Same AC signal in all three — only the DC pedestal changes.")
print("Note: Example 3 clips below 0V! The AC swing (±1.5V) exceeds the")
print("headroom below the 1.0V bias point. In a real circuit, this would")
print("distort the signal. Choose your bias point to allow full swing.")

---
## 7. The Clamper Alternative — Capacitor + Diode

There's a completely different approach to DC level shifting: the **clamper** circuit (also called a DC restorer).

Instead of stripping DC and re-biasing, a clamper *shifts* the entire waveform so that one peak is "clamped" to a reference voltage.

```
                    C
Signal In ────┤├────┬───── Signal Out
             1µF    │
                    │
                  ──┤──
                   \│/    Diode
                    │
                   GND (or V_ref)
```

How it works:

1. On the **negative half-cycle**, the diode conducts. Current flows through the diode and charges the capacitor.
2. The capacitor charges until the negative peak of the output just reaches the diode's threshold (≈0V for an ideal diode, or −0.6V for a silicon diode).
3. On the **positive half-cycle**, the diode is reverse-biased. No current flows. The capacitor holds its charge.
4. The result: the entire waveform is shifted so that the negative peak sits at the clamping voltage.

**Key difference from the divider approach:**
- **Divider re-bias:** The bias voltage is *fixed* by the divider ratio. The AC signal rides on top. Works regardless of the AC amplitude.
- **Clamper:** The shift depends on the AC amplitude. It always puts one *peak* at the reference voltage. Change the amplitude and the DC shift changes too.

Where you see clampers:
- **CRT black level restoration:** Clamps the video signal so that "black" is always at a known voltage, regardless of image content
- **ESD protection:** Clamping diodes prevent voltages from exceeding supply rails
- **Charge pumps:** Use clamping action to generate voltages higher than the supply

In [ ]:
# --- Simulate a diode clamper circuit ---
# Idealized: diode is ideal (0V forward drop), cap is large

f = 1000
dt = 1e-6
t = np.arange(0, 5e-3, dt)  # 5 cycles

V_dc_in = 0.0   # input centered on 0V (already AC coupled, or inherently bipolar)
V_ac_peak = 2.0  # ±2V input
v_in = V_dc_in + V_ac_peak * np.sin(2 * np.pi * f * t)

# Clamper simulation: cap charges through diode on negative peaks
# Output = v_in + V_cap, where V_cap is built up by diode conduction
C = 10e-6   # large cap for minimal droop
R_load = 100e3  # load resistance (discharge path)
V_cap = 0.0
v_out = np.zeros_like(t)

for i in range(len(t)):
    v_node = v_in[i] + V_cap  # voltage at output node
    
    if v_node < 0:  # diode conducts (ideal: clamps to 0V)
        # Diode forces output to 0V, charges cap
        V_cap = -v_in[i]  # whatever it takes to make v_node = 0
        v_out[i] = 0.0
    else:
        # Diode off — cap slowly discharges through load
        V_cap -= V_cap / (R_load * C) * dt  # small droop
        v_out[i] = v_in[i] + V_cap

fig, axes = plt.subplots(2, 1, figsize=(12, 7), sharex=True)

axes[0].plot(t * 1000, v_in, color='#2980b9', linewidth=2)
axes[0].axhline(y=0, color='black', linewidth=0.5)
axes[0].axhline(y=V_ac_peak, color='gray', linestyle=':', alpha=0.5)
axes[0].axhline(y=-V_ac_peak, color='gray', linestyle=':', alpha=0.5)
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title(f'Input: ±{V_ac_peak}V sine (centered on 0V)')
axes[0].set_ylim(-3, 5)

axes[1].plot(t * 1000, v_out, color='#e74c3c', linewidth=2)
axes[1].axhline(y=0, color='black', linewidth=0.5)
axes[1].axhline(y=2 * V_ac_peak, color='gray', linestyle=':', alpha=0.5,
                label=f'Positive peak: ~{2*V_ac_peak}V')
axes[1].axhline(y=0, color='orange', linestyle='--', alpha=0.5,
                label='Negative peak clamped to 0V')
axes[1].set_ylabel('Voltage (V)')
axes[1].set_xlabel('Time (ms)')
axes[1].set_title(f'Output: negative peak clamped to 0V → signal is 0V to {2*V_ac_peak}V')
axes[1].set_ylim(-3, 5)
axes[1].legend(loc='upper right')

plt.tight_layout()
plt.show()

print(f"Input:  {-V_ac_peak}V to {V_ac_peak}V (centered on 0V)")
print(f"Output: ~0V to ~{2*V_ac_peak}V (negative peak clamped to 0V)")
print(f"The whole waveform shifted up by {V_ac_peak}V — the capacitor holds this voltage.")
print(f"\nCompare with re-bias approach: re-bias sets a FIXED DC level;")
print(f"the clamper shift DEPENDS on the signal amplitude.")

---
## 8. Comparison: Re-bias vs Clamper

Both circuits shift DC levels, but they work differently and are suited to different jobs.

In [ ]:
# --- Side-by-side: re-bias vs clamper for two different input amplitudes ---

f = 1000
t = np.linspace(0, 3 / f, 3000)

amplitudes = [1.5, 0.5]  # two different AC peak values
V_bias = 2.5  # divider bias point

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

for col, V_ac in enumerate(amplitudes):
    v_ac = V_ac * np.sin(2 * np.pi * f * t)
    
    # Re-bias: fixed DC + AC
    v_rebias = V_bias + v_ac
    
    # Clamper: negative peak at 0V, so shift up by V_ac
    v_clamp = v_ac + V_ac  # idealized: negative peak at 0V
    
    axes[0, col].plot(t * 1000, v_rebias, color='#2980b9', linewidth=2)
    axes[0, col].axhline(y=V_bias, color='orange', linestyle='--', alpha=0.7,
                         label=f'DC = {V_bias}V (always)')
    axes[0, col].set_title(f'Re-bias (divider at {V_bias}V), AC = ±{V_ac}V', fontsize=11)
    axes[0, col].set_ylim(-1, 5)
    axes[0, col].set_ylabel('Voltage (V)')
    axes[0, col].legend(fontsize=9, loc='upper right')
    
    axes[1, col].plot(t * 1000, v_clamp, color='#e74c3c', linewidth=2)
    axes[1, col].axhline(y=V_ac, color='orange', linestyle='--', alpha=0.7,
                         label=f'DC = {V_ac}V (amplitude-dependent!)')
    axes[1, col].set_title(f'Clamper (neg peak at 0V), AC = ±{V_ac}V', fontsize=11)
    axes[1, col].set_ylim(-1, 5)
    axes[1, col].set_ylabel('Voltage (V)')
    axes[1, col].set_xlabel('Time (ms)')
    axes[1, col].legend(fontsize=9, loc='upper right')

plt.suptitle('Re-bias (top) vs Clamper (bottom) — same AC, different behavior',
             fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

print("Re-bias: DC level stays at 2.5V regardless of AC amplitude.")
print("Clamper: DC level changes with AC amplitude (it always puts the")
print("         negative peak at 0V, so the 'center' depends on amplitude).")

---
## 9. Comparison Table

| Method | How it works | DC result | AC result | Pros | Cons |
|--------|-------------|-----------|-----------|------|------|
| **AC couple only** | Series cap + R to GND | 0V | Preserved | Simple, one R | Can only center on 0V |
| **AC couple + divider** | Series cap + R divider from supply | Divider voltage (fixed) | Preserved (if R large enough) | Any DC level | Loading tradeoff; needs supply rail |
| **Clamper (cap + diode)** | Cap charges through diode | Peak clamped to V_ref | Preserved | No loading from divider | DC depends on amplitude; diode drop |
| **DC couple (direct wire)** | No capacitor | Original DC | Original AC | No filtering at all | Can't change DC level |

---
## 10. Build It in Falstad

### Circuit 1: AC Couple + Resistor Divider

1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Place an **AC Voltage Source** — set to 1000 Hz, Max Voltage 1.5V, DC Offset 1.5V (simulates the Fnirsi's 0–3V output).
3. Place a **1µF capacitor** in series with the source output.
4. From the capacitor's output node, place **two 10kΩ resistors in series** going to ground.
5. Connect the junction between the two resistors to a **5V DC voltage source** (positive terminal). Connect the DC source's negative terminal to ground.
6. Wait — that's wrong. The divider goes from 5V *through R1* to the output node, then *through R2* to ground. So:
   - R1 goes from the 5V source to the output node (same node where the cap connects)
   - R2 goes from the output node to ground
7. Right-click the output node wire → View in New Scope.

**What you should see:** A sine wave centered on 2.5V, swinging from 1.0V to 4.0V.

**Try:** Change the DC offset on the AC source. The output's DC level doesn't change — it stays at 2.5V. Only the AC component passes through.

### Circuit 2: Clamper

1. Place an AC Voltage Source (1000 Hz, ±2V, no DC offset).
2. Place a **1µF capacitor** in series.
3. Place a **diode** from the output node to ground (cathode at output, anode at ground — so it conducts when the output goes negative).
4. Add a **100kΩ load resistor** from the output to ground.

**What you should see:** The sine wave shifted so the negative peak sits near 0V. Output swings from ~0V to ~4V.

---
## 11. Experiment 1: Build the Re-Bias Circuit

**Goal:** Build the AC coupling + resistor divider circuit on a breadboard and observe the DC level shift.

**You'll need the AC coupling circuit from Bonus Lab 00** as a starting point — then add the divider.

**Breadboard layout:**
```
      5V USB supply
           │
          R1 = 10kΩ
           │
           ├──────────────────── Scope probe tip (CH1)
           │
Fnirsi ──┤├── Node A
        1µF   │
             R2 = 10kΩ
              │
             GND ─── Scope GND clip
                 ─── Fnirsi GND
                 ─── USB GND
```

**Important:** All grounds must be connected together — Fnirsi signal ground, scope ground clip, and USB supply ground.

**Steps:**

1. Start with the AC coupling circuit from Bonus Lab 00 (1µF cap + 10kΩ to GND).
2. **Remove the 10kΩ resistor to ground** — this was the single-resistor bias to 0V.
3. **Add two 10kΩ resistors** as a divider: one from the 5V USB supply to Node A, one from Node A to ground.
4. Connect the scope probe to Node A. Set scope to **DC coupling**, 1V/div.
5. Generate **1 kHz sine, 3V amplitude** on the Fnirsi.

**What you should see:**
- A sine wave centered on approximately **2.5V**
- Swinging from about **1.0V to 4.0V**
- V_pp ≈ 3V — same as the input

**Verify with the multimeter:**
- MM300 in **DC V mode** across Node A and GND: should read approximately **2.5V** (the DC bias)
- MM300 in **AC V mode** across Node A and GND: should read approximately **1.06V** (same as Bonus Lab 00)

---
## 12. Experiment 2: Change the Bias Point

**Goal:** Shift the bias to a different voltage by changing the divider ratio.

**Steps:**

1. Keep the circuit from Experiment 1.
2. **Replace R1** (the resistor from 5V to Node A) with **three or four 10kΩ resistors in series** (30kΩ or 40kΩ). Keep R2 = 10kΩ.
3. Observe the scope.

**With R1 = 30kΩ, R2 = 10kΩ:**

$$V_{bias} = 5\text{V} \times \frac{10\text{k}}{30\text{k} + 10\text{k}} = 1.25\text{V}$$

The sine wave should now be centered on ~1.25V, swinging from about −0.25V to 2.75V.

**With R1 = 40kΩ, R2 = 10kΩ:**

$$V_{bias} = 5\text{V} \times \frac{10\text{k}}{40\text{k} + 10\text{k}} = 1.0\text{V}$$

Center on ~1.0V, swinging from −0.5V to 2.5V. Notice the negative excursion goes below 0V — in a real circuit with an amplifier or ADC, this could cause clipping or damage.

**Key observation:** The AC amplitude (V_pp ≈ 3V) should be unchanged in all cases. Only the DC level shifted.

---
## 13. Experiment 3: Observe Loading

**Goal:** See how divider resistance affects the AC signal.

**Steps:**

1. Go back to the equal-resistor divider (R1 = R2 = 10kΩ, bias at 2.5V).
2. Measure V_pp on the scope. It should be close to 3V.
3. **Replace both 10kΩ resistors with 100kΩ resistors.** The bias voltage is the same (still V_supply/2 = 2.5V), but the divider impedance is now 50kΩ instead of 5kΩ.
4. Measure V_pp again. It should still be close to 3V — with a 1µF cap at 1kHz ($X_C$ = 159Ω), even 50kΩ is much larger.
5. **Now connect the MM300 across the output** (in DC V mode). The multimeter's 10MΩ input impedance is a very light load — the DC reading shouldn't change much. But note: with 100kΩ divider resistors, a lower-impedance load would shift the bias.

**The takeaway:** For high-impedance loads (oscilloscope, meter), you have wide freedom in choosing divider resistance. For low-impedance loads (speaker, LED, logic input), the divider needs to be stiff enough to maintain its voltage.

---
## 14. Where You See This in Real Circuits

**Amplifier interstage coupling.** A multi-stage amplifier (like a guitar amp) has coupling capacitors between each gain stage. Each stage has its own DC bias point set by resistor dividers. The AC signal hops from stage to stage through the caps, picking up a new DC level at each one. A 4-stage amplifier has 3 coupling capacitors (between stages 1–2, 2–3, and 3–4).

**ADC input conditioning.** Many ADCs expect their input centered at V_ref/2. If your sensor outputs 0–3.3V, you AC-couple it and re-bias to V_ref/2. The ADC sees the AC signal riding on its preferred DC level.

**Audio circuits.** Every stage in an audio signal chain — preamplifier, tone controls, power amplifier — uses coupling caps and bias dividers. When you see schematics with capacitors between stages and resistors to the supply rail, this is exactly what's happening.

**Oscilloscope AC coupling.** Your scope's AC coupling mode is a simplified version of this circuit: a series cap (to block DC) and the 1MΩ input impedance (as the resistor to ground, biasing the input to 0V). If the scope's front-end amplifier needed a different bias voltage, there would be a divider instead of a single resistor.

**Op-amp single-supply circuits.** Op-amps ideally want their inputs at mid-supply. In single-supply designs (0V and +5V, no negative rail), a resistor divider creates a "virtual ground" at 2.5V. The AC signal is referenced to this mid-rail instead of true ground.

---
## Summary

| Concept | Key Point |
|---------|----------|
| AC couple + divider | Capacitor strips old DC; resistor divider sets new DC; AC rides on top |
| Why it works | Superposition: cap is open at DC (divider sets level), low-Z at AC (signal passes) |
| Divider equation | $V_{bias} = V_{supply} \times R_2 / (R_1 + R_2)$ |
| Loading tradeoff | Divider R must be ≫ $X_C$ (preserve AC) and ≪ $R_{load}$ (stiff bias) |
| Clamper alternative | Cap + diode shifts waveform so one peak is at a reference voltage |
| Re-bias vs clamper | Re-bias: fixed DC, independent of amplitude. Clamper: DC depends on amplitude |
| Real-world use | Amplifier stages, ADC inputs, audio circuits, scope AC coupling |

---
## Checkpoint Questions

**Q1:** You have a 0–5V square wave and need it centered on 1.5V. What DC level does the divider need to provide? What's the output voltage range?

**Q2:** Your divider uses 1kΩ resistors and the signal source has a 10kΩ output impedance driving a 1µF coupling cap. Will the AC signal be attenuated significantly at 1 kHz? Why or why not?

**Q3:** A clamper clamps the negative peak to 0V. If the input is a ±2V sine, what's the output voltage range?

**Q4:** Why can't a capacitor alone *add* a DC offset to a signal?

**Q5:** An audio amplifier has 4 gain stages, each biased at 4.5V from a 9V supply. How many coupling capacitors are there between stages?

In [ ]:
# --- Checkpoint Answers (run to reveal) ---

print("=" * 60)
print("CHECKPOINT ANSWERS")
print("=" * 60)

# Q1
print(f"\nQ1: The 0–5V square wave has DC = 2.5V, AC = ±2.5V.")
print(f"    After AC coupling, you have ±2.5V around 0V.")
print(f"    To center on 1.5V, the divider provides V_bias = 1.5V.")
print(f"    Output range: 1.5V − 2.5V = −1.0V  to  1.5V + 2.5V = 4.0V.")
print(f"    So: −1.0V to 4.0V (note the negative excursion!).")

# Q2
R_div = 1e3
R_par = R_div / 2  # 500Ω for equal divider
C = 1e-6
f = 1000
Xc = 1 / (2 * np.pi * f * C)
# The AC signal sees Xc in series, then R_parallel to ground
# But also the 10kΩ source impedance is in series with Xc
Z_series = np.sqrt((10e3)**2 + Xc**2)  # source R + cap (approximately R_source since Xc << R_source)
gain = R_par / np.sqrt(R_par**2 + Z_series**2)
print(f"\nQ2: The divider's parallel resistance is {R_par:.0f}Ω.")
print(f"    The source impedance is 10kΩ (much larger than Xc = {Xc:.0f}Ω).")
print(f"    The AC signal sees a voltage divider: {R_par:.0f}Ω / (10kΩ + {R_par:.0f}Ω).")
print(f"    AC gain ≈ {R_par:.0f} / {10e3 + R_par:.0f} = {R_par/(10e3 + R_par):.3f} = {R_par/(10e3 + R_par)*100:.1f}%.")
print(f"    Yes — severely attenuated! The 1kΩ divider loads down the")
print(f"    high-impedance source. Use 10kΩ or 100kΩ divider resistors instead.")

# Q3
print(f"\nQ3: Input is ±2V (−2V to +2V).")
print(f"    Clamper shifts the negative peak to 0V.")
print(f"    Output: 0V to 4V (shifted up by 2V).")

# Q4
print(f"\nQ4: A capacitor blocks DC — it charges up to the average voltage")
print(f"    and then only passes deviations from that average (AC).")
print(f"    It cannot CREATE a DC voltage from nothing. To set a DC level,")
print(f"    you need a DC source — either a supply rail through a resistor")
print(f"    divider, or a charged capacitor held by a diode (clamper).")
print(f"    The capacitor alone just removes DC; it doesn't add it.")

# Q5
print(f"\nQ5: 4 gain stages need coupling caps BETWEEN each pair of stages.")
print(f"    Between stages: 1→2, 2→3, 3→4 = 3 coupling capacitors.")
print(f"    (There may also be a coupling cap at the input and/or output,")
print(f"    but the question asks about between stages: 3.)")